# **Homework 5 - Visit the Wikipedia hyperlinks graph!**

In [1]:
from collections import defaultdict
import networkx as nx
import pandas as pd
import operator
import math
import os 
import json
import numpy as np
import functions as fn
import importlib
importlib.reload(fn)

<module 'functions' from 'C:\\Users\\user\\ADM\\hw5\\Hw5-Gr10-master\\functions.py'>

- We create a graph from the 'wiki-topcats-categories.txt' file. 
- From all categories, we will only take into account those with at least 3500 articles. 


In [2]:
G, cat_dict = fn.create_graph()

- Remove all nodes with no edges. 
- Re-check all categories have more than 3500 articles

In [3]:
#remove nodes with no edges and keep only categories with more than 3500 nodes
for c in cat_dict: 
    for nodes in cat_dict[c]: 
        if len(list(nx.neighbors(G,nodes))) == 0: 
            #G.remove_node(g)
            cat_dict[c].remove(nodes)            
cat_dict = {c: cat_dict[c] for c in cat_dict if len(cat_dict[c]) >= 3500}

cat_dict contains the category as key and the nodes as value

We have a directed graph. 
- Number of nodes:  $n$

In [5]:
G.number_of_nodes()

546237

- Number of edges: $m$

In [6]:
G.size()

2645247

- Average number of degree: $$ \frac{m}{n} $$

In [7]:
G.size() / G.number_of_nodes()

4.842672686031888

- Graph density:
$$\frac{m}{n(n-1)}$$

In [8]:
nx.density(G)

8.865531905681586e-06

## Shortest Path 

Our task here is to find the shortest path from C0 to Ci; where C0 is an input category and Ci is the rest of the categories. We have used Dijkstra's algorithm.

Running the shortest path algorithm for all the nodes and categories takes too much time. Therefore we have selected a small portion of it to show how the algorithm works. 
- We will take into account 10 categories and 500 nodes from each. For maximum effect we have choosen nodes with high links. 

In [45]:
cat_ten = fn.ten_category(cat_dict)

In [23]:
with open('category_10.json')as f: 
    cat_ten = json.load(f)

An article may belong to multiple categories,
- If the article belongs to the input cateogry and other categories, it belongs to that input category
- Other wise it belongs to the category closest to the input category

The first category is considered as the input category

In [10]:
cat_ten = fn.remove_duplicates(cat_ten)

Set up for the shortest path algorithm. 

In [11]:
with open('category_10.json')as f: 
    cat_ten = json.load(f)
temp = []
for c in list(cat_ten.keys()): 
    temp.append(cat_ten[c])    
destination_nodes = []
for i in temp:
    for n in i: 
        destination_nodes.append(n)
initial_nodes = cat_ten['English_footballers']

We have removed all nodes with no edges. But as mentioned above, due to time constraint we have only considered a few portion of the entire graph and some nodes may not have a link with in the nodes choosen. 

In [ ]:
fn.find_shortest_path(initial_nodes, destination_nodes)

In [3]:
with open('dict.json')as f: 
    dic = json.load(f)

dic is the result of the shortest path algorithm

In [44]:
fn.computeDist('82393','83651',dic)

3

## Block ranking

Our task is to sort the categories based on the median of all possible shortest path between input category and other categories. 

- Here we find the median between our categories. We use `shortest_path` function, which returns all possible shortest paths within a category, and the `distance` function finds the median of the shortest paths. 

In [47]:
categories = list(cat_ten.keys())
median_distance = defaultdict(list)
idx = 0
C0 = categories[0]
for Ci in categories:
    if Ci != C0:
        median_distance[Ci] = fn.distance(C0, Ci, cat_ten, dic)  
        print(Ci, median_distance[Ci])
        with open("median.json", "w") as f:
            json.dump(median_distance, f)    

The_Football_League_players 4.0
Association_football_forwards 4.0
Association_football_goalkeepers 5.0
Association_football_midfielders 4.0
Association_football_defenders 5.0
Living_people 6.0
Harvard_University_alumni inf
Major_League_Baseball_pitchers 8.0
Members_of_the_United_Kingdom_Parliament_for_English_constituencies 5.0


- Sort the categories in ascending order based on median

In [48]:
sorted_median = sorted(median_distance.items(), key=operator.itemgetter(1), reverse=True)     
sorted_median

[('Harvard_University_alumni', inf),
 ('Major_League_Baseball_pitchers', 8.0),
 ('Living_people', 6.0),
 ('Association_football_goalkeepers', 5.0),
 ('Association_football_defenders', 5.0),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 5.0),
 ('The_Football_League_players', 4.0),
 ('Association_football_forwards', 4.0),
 ('Association_football_midfielders', 4.0)]

After we obtain the block ranking, we need to sort the nodes inside each category based on number of edges. 

- Calculate the number of edges of each node in each category

In [57]:
G2 = nx.DiGraph()
edges = G.edges()
for Ci in cat_ten:    
    fn.update_score(G2, edges, Ci, cat_ten)

Sort nodes based on score(number of in degree) $$(node, score)$$

In [71]:
scores = sorted(nx.get_node_attributes(G2,'score').items(), key=lambda x: -x[1])

Loop throug our block ranking and print the sorted nodes from each category

In [72]:
rank = fn.rank(sorted_median, scores, cat_ten)

creating the dictionary with the names of the nodes (articles) of wikipedia

In [73]:
page_names = {}

with open('wiki-topcats-page-names.txt') as f: 
    for line in f:
        l = line.strip().split()
        page_names[l[0]] = ' '.join(l[1:])

printing the first 100 articles in the rank

In [74]:
for i in range(100):
    print(page_names[str(rank[i])])

John F. Kennedy
Al Gore
David Davis (British politician)
Theodore Roosevelt
Michael Dukakis
Henry Kissinger
Leonard Bernstein
Robert F. Kennedy
Jack Lemmon
John Adams
Conan O'Brien
T. S. Eliot
Lawrence Summers
Jacques Chirac
Natalie Portman
John Quincy Adams
John Ashbery
Yvette Cooper
Terrence Malick
Darren Aronofsky
William Waldegrave, Baron Waldegrave of North Hill
Cole Porter
Ban Ki-moon
Mark Zuckerberg
Pierre Trudeau
Norman Mailer
John Lithgow
Frank O'Hara
Jacques Derrida
Michael Ignatieff
Jeffrey Sachs
Yo-Yo Ma
William Weld
James Bryant Conant
Henry Cabot Lodge
Mira Sorvino
Cornel West
David Rockefeller
Steven Pinker
George Plimpton
James Russell Lowell
Pete Seeger
Daniel Dennett
Saul Kripke
Walter Piston
Paul Samuelson
Randall Thompson
Tom Lehrer
Robert Lowell
William Randolph Hearst
Robert Bly
Kenneth Koch
Samuel Adams
Ben Bernanke
Peter Benchley
Henry Cabot Lodge, Jr.
Archibald Cox
Charles Sanders Peirce
Charles Sumner
Abbott Lawrence Lowell
Hilary Putnam
A. O. Scott
Donald Dav